In [66]:
import pandas as pd
import numpy as np
from matplotlib import rc
from matplotlib import pyplot as plt
import seaborn as sns
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('../GP_전처리완료/GP_통합(3years)_음수값제거.csv')

In [5]:
emp_df = df.iloc[:,1:-1]
target = df.iloc[:,-1]

In [125]:
# 비선형 PCA (커널 pca)
from sklearn.decomposition import KernelPCA

kpca = KernelPCA(n_jobs=-1)
features_kpca = kpca.fit_transform(emp_df)

features_kpca

In [94]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### 로지스틱 회귀와 KNN 기반 소프트 보팅

In [116]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()
vo_clf = VotingClassifier(estimators = [('LR', lr_clf), ('KNN', knn_clf)], voting='soft')

X_train, X_test, y_train, y_test = train_test_split(emp_df, target, test_size=0.3)

In [117]:
classifiers = [vo_clf, lr_clf, knn_clf]

for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print(classifier.__class__.__name__, ':', accuracy_score(y_test, pred))

VotingClassifier : 0.9943672319843262
LogisticRegression : 0.9994489683462928
KNeighborsClassifier : 0.9633870078981204


In [106]:
vo_clf.named_estimators_

{'LR': LogisticRegression(), 'KNN': KNeighborsClassifier()}

In [108]:
vo_clf.classes_

array([0, 1])

In [109]:
vo_clf.n_features_in_

72

### 로지스틱 회귀와 KNN 기반 하드 보팅

In [115]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()
vo_clf = VotingClassifier(estimators = [('LR', lr_clf), ('KNN', knn_clf)], voting='hard')

X_train, X_test, y_train, y_test = train_test_split(emp_df, target, test_size=0.3)

classifiers = [vo_clf, lr_clf, knn_clf]

for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    print(classifier.__class__.__name__, ':', accuracy_score(y_test, pred))

VotingClassifier : 0.9763056388905896
LogisticRegression : 0.9998163227820976
KNeighborsClassifier : 0.9629584277230148


### 랜덤 포레스트

In [118]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [121]:
params = {'n_estimators' : [10, 20, 30, 40, 50],
        'max_depth' : [2, 3, 4, 5],
        'min_samples_leaf' : [4, 5, 6, 7],
        'min_samples_split' : [8, 16, 20]}

In [122]:
X_train, X_test, y_train, y_test = train_test_split(emp_df, target, test_size=0.3)

rf_clf = RandomForestClassifier(n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 10}
1.0
